# Deep Learning with PyTorch: A 60 Minute Blitz

Goal of this tutorial:
- Understand PyTorch’s Tensor library and neural networks at a high level.
- Train a small neural network to classify images.

## Part 3: Neural Networks

Neural networks can be constructed using the <b>torch.nn</b> package.

Now that you had a glimpse of <b>autograd</b>, <b>nn</b> depends on <b>autograd</b> to define models and differentiate them. An <b>nn.Module</b> contains layers, and a method <b>forward(input)</b> that returns the <b>output</b>.

A typical training procedure for a neural network is as follows:
- Define the neural network that has some learnable parameters (or weights).
- Iterate over a dataset of inputs.
- Process input through the network.
- Compute the loss (how far is the output from being correct).
- Propagate gradients back into the network’s parameters.
- Update the weights of the network, typically using a simple update rule: <b>weight = weight - learning_rate * gradient</b>

### Defien the Network

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square Conv kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120) # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # if the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [3]:
net = Net()
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

You just have to define the <b>forward</b> function, and the <b>backward</b> function (where gradients are computed) is automatically defined for you using <b>autograd</b>. You can use any of the Tensor operations in the <b>forward</b> function.

The learnable parameters of a model are returned by <b>net.parameters()</b>:

In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # Conv1's weight

10
torch.Size([6, 1, 3, 3])


Let’s try a random 32x32 input. Note: expected input size of this net (LeNet) is 32x32. To use this net on the MNIST dataset, please resize the images from the dataset to 32x32.

In [5]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
out

tensor([[-0.1420, -0.0469,  0.0183, -0.0682,  0.0007,  0.0362, -0.0687,  0.0489,
         -0.0638,  0.1042]], grad_fn=<AddmmBackward>)

Zero the gradient buffers of all parameters and backprops with random gradients:

In [6]:
net.zero_grad()
out.backward(torch.randn(1, 10))

<b>Notes:</b>
- <b>torch.nn</b> only supports mini-batches. The entire <b>torch.nn</b> package only supports inputs that are a mini-batch of samples, and not a single sample.
- For example, <b>nn.Conv2d</b> will take in a 4D Tensor of <b>nSamples x nChannels x Height x Width</b>.
- If you have a single sample, just use <b>input.unsqueeze(0)</b>
to add a fake batch dimension.

<b> Recap:</b>
- <b>torch.Tensor</b> - A multi-dimensional array with support for autograd operations like <b>backward()</b>. Also holds the gradient w.r.t. the tensor.
- <b>nn.Module</b> - Neural network module. Convenient way of encapsulating parameters, with helpers for moving them to GPU, exporting, loading, etc.
- <b>nn.Parameter</b> - A kind of Tensor, that is automatically registered as a parameter when assigned as an attribute to a <b>Module</b>.
- <b>autograd.Function</b> - Implements forward and backward definitions of an autograd operation. Every Tensor operation creates at least a single <b>Function</b> node that connects to functions that created a <b>Tensor</b> and encodes its history.

### Loss Function

A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.

There are several different loss functions under the nn package . A simple loss is: <b>nn.MSELoss</b> which computes the mean-squared error between the input and the target.

In [7]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
loss

tensor(0.8271, grad_fn=<MseLossBackward>)

When we call <b>loss.backward()</b>, the whole graph is differentiated w.r.t. the loss, and all Tensors in the graph that has <b>requires_grad=True</b> will have their <b>.grad</b> Tensor accumulated with the gradient.

In [8]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

### Backprop

To backpropagate the error all we have to do is to <b>loss.backward()</b>. You need to clear the existing gradients though, else gradients will be accumulated to existing gradients.

Now we shall call <b>loss.backward()</b>, and have a look at conv1’s bias gradients before and after the backward.

In [9]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0049, -0.0059,  0.0007,  0.0056,  0.0098,  0.0042])


### Update the Weights

The simplest update rule used in practice is the Stochastic Gradient Descent (SGD):
weight = weight - learning_rate * gradient

We can implement this using simple Python code:

In [10]:
lr = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * lr)

However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. To enable this, we built a small package: <b>torch.optim</b> that implements all these methods. Using it is very simple:

In [15]:
# create the optimizer:
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

# in the training loop:
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # dose the update